<a href="https://colab.research.google.com/github/Tandon-A/SPAIC-Udacity/blob/master/PATE_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install syft

In [2]:
import torch.nn as nn
from collections import OrderedDict
import numpy as np
import torch
from torch import optim
from torchvision import datasets, transforms



#Getting MNIST Data

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5), (0.5)),
                              ])
mnist_trainset = datasets.MNIST(root='./train_data', train=True, download=True, transform=transform)

trainX = mnist_trainset.train_data
trainY = mnist_trainset.train_labels
testX = mnist_trainset.test_data
testY = mnist_trainset.test_labels
print (trainX.shape,trainY.shape,testX.shape,testY.shape)

#Shuffling the data
perm = np.random.permutation(len(trainX))
trainX = trainX[perm,:]
trainY = trainY[perm]
perm = np.random.permutation(len(testX))
testX = testX[perm,:]
testY = testY[perm]

0it [00:00, ?it/s]

9920512it [00:01, 8950179.31it/s]                            


Extracting ./train_data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 135382.23it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./train_data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2438987.99it/s]                           
0it [00:00, ?it/s]

Extracting ./train_data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 51075.54it/s]            
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Extracting ./train_data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!
torch.Size([60000, 28, 28]) torch.Size([60000]) torch.Size([60000, 28, 28]) torch.Size([60000])


In [0]:
#Defining model architecture
model = nn.Sequential(OrderedDict([
    ('fc1',nn.Linear(784,512)),
    ('r1',nn.ReLU()),
    ('dropout1',nn.Dropout(p=0.2)),
    ('fc2',nn.Linear(512,256)),
    ('r2',nn.ReLU()),
    ('dropout2',nn.Dropout(p=0.2)),
    ('fc3',nn.Linear(256,64)),
    ('r3',nn.ReLU()),
    ('fc4',nn.Linear(64,10)),
    ('log_ps',nn.LogSoftmax(dim=1))
]))


criterion = nn.NLLLoss() 
opt = optim.Adam(model.parameters(),lr= 0.001)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
num_teachers = 30   #number of teacher models
subset_size = len(trainX) // num_teachers #data per teacher model

valdata = torch.utils.data.TensorDataset(testX,testY)
valloader = torch.utils.data.DataLoader(valdata,batch_size = 50, shuffle = False)
train_pred = np.zeros((len(testX),num_teachers)) #array to save teacher model predictions

#Function to reset model parameters
def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()


for i in range(num_teachers):  
  model.apply(weight_reset)
  model = model.to(device)
  #Creating DataLoader for a specific teacher model
  tensor_dataset = torch.utils.data.TensorDataset(trainX[i*subset_size:(i+1)*subset_size],trainY[i*subset_size:(i+1)*subset_size])
  trainloader_teacher = torch.utils.data.DataLoader(tensor_dataset,batch_size=50,shuffle=True)
  epochs = 30  #training each teacher model for 30 epochs 
  for e in range(epochs):
    running_loss = 0
    model.train() #setting train mode
    for images,labels in iter(trainloader_teacher):
      images = images.view(images.shape[0],-1).float().to(device)#flattening the images and then sending them to corresponding device
      labels = labels.to(device)
      opt.zero_grad()
      log_ps = model(images)
      loss = criterion(log_ps,labels)
      loss.backward()
      opt.step()
      running_loss += loss
  with torch.no_grad():
    #Teacher Model test after training 
    model.eval()
    ps = torch.exp(model(testX.view(testX.shape[0],-1).float().to(device)))
    topp,topk = ps.topk(1,dim=1)
    equals = topk == testY.view(*topk.shape).to(device)
    accuracy = torch.mean(equals.type(torch.FloatTensor))    
    print ('teacher =%r accuracy =%r ' %(i,accuracy))
    train_pred[:,i] = np.argmax(ps.cpu(),axis=1)#saving teacher predictions
print ('completed ',train_pred.shape)

teacher =0 accuracy =tensor(0.9165) 
teacher =1 accuracy =tensor(0.9152) 
teacher =2 accuracy =tensor(0.9237) 
teacher =3 accuracy =tensor(0.9306) 
teacher =4 accuracy =tensor(0.9253) 
teacher =5 accuracy =tensor(0.9240) 
teacher =6 accuracy =tensor(0.9294) 
teacher =7 accuracy =tensor(0.9293) 
teacher =8 accuracy =tensor(0.9265) 
teacher =9 accuracy =tensor(0.9177) 
teacher =10 accuracy =tensor(0.9316) 
teacher =11 accuracy =tensor(0.9184) 
teacher =12 accuracy =tensor(0.9175) 
teacher =13 accuracy =tensor(0.9277) 
teacher =14 accuracy =tensor(0.9252) 
teacher =15 accuracy =tensor(0.9270) 
teacher =16 accuracy =tensor(0.9255) 
teacher =17 accuracy =tensor(0.9288) 
teacher =18 accuracy =tensor(0.9183) 
teacher =19 accuracy =tensor(0.9151) 
teacher =20 accuracy =tensor(0.9234) 
teacher =21 accuracy =tensor(0.9168) 
teacher =22 accuracy =tensor(0.9245) 
teacher =23 accuracy =tensor(0.9222) 
teacher =24 accuracy =tensor(0.9139) 
teacher =25 accuracy =tensor(0.9078) 
teacher =26 accuracy =

In [6]:
#generating labels for student models by adding noise to teacher model predictions
new_labels_noise = list()
for i in range(len(train_pred)):
  label_count = np.bincount(train_pred[i,:].astype(int),minlength=10)
  eps = 0.1
  beta = 1/eps
  for k in range(len(label_count)):
    label_count[k] += np.random.laplace(0,beta,1)
  new_labels_noise.append(np.argmax(label_count))
new_labels_noise = torch.Tensor(np.array(new_labels_noise)).type(torch.LongTensor)
print (new_labels_noise.shape)

torch.Size([60000])


In [7]:
#Preparing training data for student models by using test data and new noise labels
tensor_train_dataset = torch.utils.data.TensorDataset(testX[0:50000],new_labels_noise[0:50000])
new_teacher_dataloader = torch.utils.data.DataLoader(tensor_train_dataset,batch_size=50,shuffle=True)

model.apply(weight_reset)
model = model.to(device)
epochs = 30
for e in range(epochs):    
  running_loss = 0
  model.train() #Training student model
  for images,labels in iter(new_teacher_dataloader):
    images = images.view(images.shape[0],-1).float().to(device)
    labels = labels.to(device)
    opt.zero_grad()
    log_ps = model(images)
    loss = criterion(log_ps,labels)
    loss.backward()
    opt.step()
    running_loss += loss
with torch.no_grad():
  model.eval()
  #Testing student model accuracy with original labels 
  ps = torch.exp(model(testX[50000:].view(testX[50000:].shape[0],-1).float().to(device)))
  topp,topk = ps.topk(1,dim=1)
  equals = topk == testY[50000:].view(*topk.shape).to(device)
  accuracy = torch.mean(equals.type(torch.FloatTensor))  
  print ('student testY accuracy =%r ' %(accuracy))

student testY accuracy =tensor(0.9237) 


In [8]:
from syft.frameworks.torch.differential_privacy import pate

data_dep_eps, data_indep_eps = pate.perform_analysis(teacher_preds = torch.Tensor(train_pred[59000:].T).type(torch.LongTensor), indices = new_labels_noise[59000:], noise_eps = 0.1, delta =1e-5,moments=10)
print (data_dep_eps,data_indep_eps)

51.512925464969555 51.51292546497024
